In [11]:
# using Distributions 

# abstract type Distribution{T} end
# struct Half_Cauchy <: Distribution{Float64} end
# const half_cauchy = Half_Cauchy()

# function logpdf(::Half_Cauchy, x::Real, x0::Real, gamma::Real)
#     if x<0
#             x = x*-1
#     end
            
#     return Distributions.logpdf(Distributions.Cauchy(x0, gamma), x)
    
# end

# function logpdf_grad(::Half_Cauchy, x::Real, x0::Real, gamma::Real)
#     if x<0
#             x = x*-1
#     end
            
#     x_x0 = x - x0
#     x_x0_sq = x_x0^2
#     gamma_sq = gamma^2
#     deriv_x0 =  2 * x_x0 / (gamma_sq + x_x0_sq)
#     deriv_x = - deriv_x0
#     deriv_gamma = (x_x0_sq - gamma_sq) / (gamma * (gamma_sq + x_x0_sq))
#     (deriv_x, deriv_x0, deriv_gamma)
# end

# is_discrete(::Half_Cauchy) = false

# random(::Half_Cauchy, x0::Real, gamma::Real) = abs(rand(Distributions.Cauchy(x0, gamma)))

# (::Half_Cauchy)(x0::Real, gamma::Real) = random(Half_Cauchy(), x0, gamma)

# has_output_grad(::Half_Cauchy) = true
# has_argument_grads(::Half_Cauchy) = (true, true)

# export half_cauchy


In [12]:
# using  Gen

# @gen function a()
#     x ~ half_cauchy(0,1)

# end

# traces = [simulate(a, ()) for _ in 1:2]  # Run 100 importance samples

In [22]:
using Gen
using Plots

using Distributions 




#my_dist = print(rand(half_cauchy(0,25)))
#my_dist = Truncated(Cauchy(), 0, Inf)





# Define the model with dynamic eta sampling
@gen function simple_normal_model(sigma)
    mu ~ normal(0, 1)     # Sample mu from a normal distribution
    tau  ~ normal(2, 5)   # Sample tau from a Half-Cauchy distribution
    
    # Dynamic eta values sampled from normal distributions
    list_of_Eta = [{(:eta, i)} ~ normal(0, 1) for i=1:length(sigma)]
    
    for i in 1:length(sigma)  # Loop over 5 iterations
        # Calculate theta based on mu, tau, and eta
        theta = mu + tau * list_of_Eta[i]
        
        # Sample obs from a normal distribution with mean theta and standard deviation sigma[i]
         {(:y, i)} ~ normal(theta, sigma[i])
    end

    
    
    
end

function do_inference(model, sigma, y_obs, num_iters, L, eps)
    
    # Create a choice map that maps model addresses (:y, i)
    # to observed values ys[i]. We leave :slope and :intercept
    # unconstrained, because we want them to be inferred.
    observations = choicemap()
    for (i, y) in enumerate(y_obs)
        
        observations[(:y, i)] = y
    end
    
    # Call importance_resampling to obtain a likely trace consistent
    # with our observations.
    (trace, _) = generate(model, (sigma,), observations);
    
    accepted = 0
  
    # Metropolis–Hastings on :logtheta only (m0 and t2 are fixed)
    for _ in 1:num_iters
        # Run HMC with specified step size and trajectory length
        (trace, accepted_this_iter) = hmc(trace, select(:mu, :tau, (:eta, i for i in 1:length(sigma))...), L=L, eps=eps)
        accepted+= accepted_this_iter
        
    end

    # Extract the final logtheta
    final_choices = get_choices(trace)
   
    final_mu = final_choices[:mu]
    final_tau = final_choices[:tau]
    final_etas = [final_choices[(:eta, i)] for i in 1:length(sigma)]

    acceptance_rate = accepted / num_iters  # Divide by total number of proposals

    return (final_mu, final_tau, final_etas, acceptance_rate)
end


y_obs = [28, 8, -3, 7, -1, 1, 18, 12]
sigma =[15, 10, 16, 11, 9, 11, 10, 18]


# Use importance sampling as an example:
# traces = [simulate(simple_normal_model, (sigma,)) for _ in 1:100]  # Run 100 importance samples

# trace = do_inference(simple_normal_model, sigma, y_obs, 1000, 1, 0.1)

## Run inference
configurations = [
    (eps=2, L=1),
    (eps=2, L=2),
    (eps=2, L=5),
    (eps=2, L=10),
    (eps=2, L=20)
]


# acceptance_rates = []
# for (eps, L) in configurations
#     final_mu, final_tau, final_etas, accepted = do_inference(simple_normal_model, sigma, y_obs, 5000, L, eps)
#     println("Configuration: eps = $eps, L = $L")
#     println("Acceptance rate: $accepted")
#     #push!(acceptance_rates, (eps, L, acceptance_rate))
# end

final_mu, final_tau, final_etas, accepted = do_inference(simple_normal_model, sigma, y_obs, 5000, 1, 2)
println("Final inferred mu: ", final_mu)
println("Final inferred tau: ", final_tau)

println("Final inferred etas: ", final_etas)

Final inferred mu: -0.03993062739567588
Final inferred tau: -3.251773625600974
Final inferred etas: [-0.39362237733627237, -0.10491281203139322, -1.5647180144604487, 0.20383097175604525, 1.5902634953952965, 0.2402140861756465, -0.12215540846954251, -0.869889725342703]


In [2]:
using Gen
using Distributions
using LinearAlgebra

@gen function linear_regression_model(X, y, N, K, sigma_alpha2, mu_beta, sigma_beta2, lambda_sigma)
    # Sample the prior for intercept alpha and regression coefficients beta
    alpha ~ normal(0, sqrt(sigma_alpha2))            # Intercept
    beta = [{(:beta, i)} ~ normal(mu_beta, sqrt(sigma_beta2)) for i in 1:K]      # Regression coefficients
    
    # Sample degrees of freedom and scale for the Student's t-distribution
    nu ~ gamma(2, 10)                                # Degrees of freedom for Student's t
    sigma ~ exponential(lambda_sigma)                 # Scale for the Student's t

    # Compute the mean for each y_i: μ_i = α + X_i * β
    for i in 1:N
       
        mu_i = alpha + dot(X[i, :], beta)           # Linear model: μ_i = α + X_i * β
        {(:y, i)} ~ student(mu_i, sigma)           # Sample y_i from Student's t distribution
    end
    
    return :y  # Return the observed responses
end


DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any, Any, Any, Any, Any, Any, Any, Any], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], Main.var"##linear_regression_model#231", Bool[0, 0, 0, 0, 0, 0, 0, 0], false)

In [3]:
# Example run function for inference (adjusted to your example)
function do_inference_linear_regression(X, y, N, K, sigma_alpha2, mu_beta, sigma_beta2, lambda_sigma, num_iters, L, eps)
    # Set up observations as a choice map
    observations = Gen.choicemap()
    for (i, y) in enumerate(y)
        observations[(:y, i)] = y
    end

    # Generate the model with initial data and priors
    (trace, _) = generate(linear_regression_model, (X, y, N, K, sigma_alpha2, mu_beta, sigma_beta2, lambda_sigma), observations)
    
    accepted = 0
  
    # Metropolis-Hastings on the parameters with HMC
    for _ in 1:num_iters
        # Run HMC with specified step size and trajectory length
      
       (trace, accepted_this_iter) = hmc(trace, select(:alpha, :nu, :sigma,(:beta, i for i in 1:K)...), L=L, eps=eps)

        accepted += accepted_this_iter
    end

    # Extract the final choices for parameters
    final_choices = get_choices(trace)
    final_alpha = final_choices[:alpha]
    final_betas = [final_choices[(:beta, i)] for i in 1:K]
    final_nu = final_choices[:nu]
    final_sigma = final_choices[:sigma]

    acceptance_rate = accepted / num_iters  # Divide by total number of proposals

    return final_alpha, final_betas, final_nu, final_sigma, acceptance_rate
end

# Example of running inference with some random data
N = 100  # Number of data points
K = 5    # Number of features
sigma_alpha2 = 1.0  # Prior variance for alpha
mu_beta = 0.0  # Mean for beta prior
sigma_beta2 = 1.0  # Variance for beta prior
lambda_sigma = 1.0  # Rate for sigma prior

# Generate synthetic data for testing
X = randn(N, K)  # N x K feature matrix
true_alpha = 2.0
true_beta = randn(K)  # True regression coefficients
nu_true = 5.0
sigma_true = 1.0

# Linear model for generating synthetic y values
mu = X * true_beta .+ true_alpha
y = randn(N) .* sigma_true .+ mu  # Add noise to y values

# Run inference
final_alpha, final_betas, final_nu, final_sigma, acceptance_rate = do_inference_linear_regression(
    X, y, N, K, sigma_alpha2, mu_beta, sigma_beta2, lambda_sigma, 5000, 5 , 0.001
)

println("Final alpha: ", final_alpha)
println("Final betas: ", final_betas)
println("Final nu: ", final_nu)
println("Final sigma: ", final_sigma)
println("Acceptance rate: ", acceptance_rate)

Final alpha: 0.3370987645106329
Final betas: [0.2662668825476579, -1.144281211677809, 0.6915114681152962, -1.142215894993264, -0.9695292029160658]
Final nu: 24.33062411187261
Final sigma: 4.446274114584083
Acceptance rate: 1.0
